# 실습 [26-1]<br>
**실습명: LSTM으로 문장 생성하기**<br>
- 네이버 제공 nsmc 영화 리뷰 데이터셋을 이용하여 모델 훈련

In [1]:
# Colab 환경에서 실행할 경우 빠른 모델 훈련을 위해 런타임 유형을 GPU로 설정하는 것이 좋다.
%tensorflow_version 2.x
import tensorflow as tf

In [2]:
tf.__version__

'2.5.0'

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import os
import time
import tensorflow_datasets as tfds

In [4]:
# tf.keras.utils.get_file 함수를 통해 데이터셋을 불러온다.
# tf.data.TextLineDataset 함수로 데이터를 읽는다. 
url = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt'
text_path = tf.keras.utils.get_file('ratings_train.txt', origin=url)  
ds_file = tf.data.TextLineDataset(text_path)

14630912/14628807 [==============================] - 0s 0us/step


In [5]:
# tfds.features.text.Tokenizer()를 이용하여 입력 텍스트를 토큰화할 수 있다.
import tensorflow_datasets as tfds
tokenizer = tfds.deprecated.text.Tokenizer()

In [6]:
print(tokenizer.tokenize("나는 매일 아침 지하철을 탄다"))

['나는', '매일', '아침', '지하철을', '탄다']


In [7]:
# 데이터셋의 구성을 확인한다.
for sample in ds_file.take(3):
  tokens = tokenizer.tokenize(sample.numpy())
  # 텍스트 부분만 출력한다.
  print(tokens[1:-1])

['document']
['아', '더빙', '진짜', '짜증나네요', '목소리']
['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나']


In [8]:
# 데이터셋에서 텍스트 부분만을 리스트에 담아 훈련에 사용할 텍스트 데이터셋을 만든다.
docs = []
with open(text_path, 'r',encoding='utf-8') as f:
  next(f)
  for line in f:
    text = line.split('\t')[1]
    docs.append(text)

print("문장 개수: ",len(docs))
print(docs[:5])

문장 개수:  150000
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']


In [9]:
# 텍스트 데이터셋에 포함되는 모든 고유 문자로 vocab을 구축한다.
whole_text = ' '.join(docs)
vocab = sorted(set(whole_text))
print("고유 문자수: {}개".format(len(vocab)))

고유 문자수: 3004개


In [10]:
# 데이터셋에 포함된 모든 문자를 숫자로 치환하여 임베딩한다. 
# 이를 위해 우선적으로 문자-인덱스 사전을 구축한다.
char2idx = {u:i for i, u in enumerate(vocab)}
idex2char = np.array(vocab)

# 텍스트 데이터셋을 숫자 벡터로 임베딩한다.
text_as_int = np.array([char2idx[c] for c in whole_text])

In [11]:
# 각 문자가 어떤 숫자로 매핑되었는지 확인할 수 있다.
print('{')
for char,_ in zip(char2idx, range(20)):
  print(' {:4s}:{:3d},'.format(repr(char), char2idx[char]))
print(' ...\n}')

{
 ' ' :  0,
 '!' :  1,
 '"' :  2,
 '#' :  3,
 '$' :  4,
 '%' :  5,
 '&' :  6,
 "'" :  7,
 '(' :  8,
 ')' :  9,
 '*' : 10,
 '+' : 11,
 ',' : 12,
 '-' : 13,
 '.' : 14,
 '/' : 15,
 '0' : 16,
 '1' : 17,
 '2' : 18,
 '3' : 19,
 ...
}


In [12]:
# 마찬가지로 데이터셋의 텍스트가 어떻게 숫자로 매핑되었는지 확인할 수 있다.
print ('입력 문장: \n{}\n\n숫자 매핑: \n{}'.format(repr(whole_text[:20]), text_as_int[:20]))

입력 문장: 
'아 더빙.. 진짜 짜증나네요 목소리 '

숫자 매핑: 
[1954    0  974 1593   14   14    0 2327 2342    0 2342 2321  789  844
 2091    0 1387 1757 1312    0]


In [13]:
# 샘플 길이와 epoch 길이를 정하고 훈련 샘플을 만든다.
seq_length = 100
examples_per_epoch = len(whole_text)//seq_length
print(examples_per_epoch)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
  print(idex2char[i.numpy()])

54356
아
 
더
빙
.


In [14]:
# 훈련 샘플을 배치 크기로 변환한다.
# 샘플 문장의 다음 문자를 예측하도록 훈련해야 하므로 배치 사이즈는 샘플 길이+1로 설정한다.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idex2char[item.numpy()])))

'아 더빙.. 진짜 짜증나네요 목소리 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 너무재밓었다그래서보는것을추천한다 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'
' 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반'
'개도 아까움. 원작의 긴장감을 제대로 살려내지못했다. 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족'
'도없다 연기못하는사람만모엿네 액션이 없는데도 재미 있는 몇안되는 영화 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? 걍인피니트가짱이다.진짜짱이다♥'
' 볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~ 울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해 담백하고 깔끔해서 좋다. 신문기사로만'


In [15]:
# 학습 샘플에서 입력 텍스트와 타깃 텍스트 부분을 명시하는 함수를 만든다.
# map 메서드를 이용해 해당 함수를 각 배치에 적용한다.
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
# 데이터셋에서 입력 텍스트와 타깃 텍스트가 잘 분리 되었는지 확인한다.
for input_example, target_example in dataset.take(1):
  print('입력 텍스트: ', repr(''.join(idex2char[input_example.numpy()])))
  print('타깃 텍스트: ', repr(''.join(idex2char[target_example.numpy()])))

입력 텍스트:  '아 더빙.. 진짜 짜증나네요 목소리 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 너무재밓었다그래서보는것을추천한다 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조'
타깃 텍스트:  ' 더빙.. 진짜 짜증나네요 목소리 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 너무재밓었다그래서보는것을추천한다 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'


In [17]:
# 모델은 임베딩 된 문자 입력이 들어올 때 다음 문자를 예측해서 출력해야 한다.
# 훈련 샘플을 통해 이를 확인할 수 있다.
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("{:4d}단계".format(i))
  print("입력: {} ({:s})".format(input_idx, idex2char[input_idx]))
  print("예상출력: {} ({:s})".format(target_idx, idex2char[target_idx]))

   0단계
입력: 1954 (아)
예상출력: 0 ( )
   1단계
입력: 0 ( )
예상출력: 974 (더)
   2단계
입력: 974 (더)
예상출력: 1593 (빙)
   3단계
입력: 1593 (빙)
예상출력: 14 (.)
   4단계
입력: 14 (.)
예상출력: 14 (.)


In [18]:
# 훈련 epoch 당 다루게 될 시퀀스 데이터를 배치 데이터로 만든다.
# 이때 모델의 일반화 능력을 높이기 위해 데이터셋을 섞는다.
# 매 epoch 당 100개 문자로 이루어진 64개의 데이터를 훈련하게 된다.
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [19]:
# 모델 설계를 위해 vocab size, embedding dimension, rnn units number를 설정한다.
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [20]:
# tf.keras.Sequential을 이용해 구성 층을 이루고 모델을 정의한다.
# tf.keras.layers.Embedding은 문자 데이터를 임베딩 벡터 상에 정수 형태로 매핑하는 임베딩 층이다.
# tf.keras.layers.LSTM은 n개의 rnn_units으로 이루어진 순환 신경망 층이다. LSTM 대신 GRU를 사용할 수도 있다.
# tf.keras.Dense는 vocab_size의 크기를 갖는 출력층으로, 출력 결과는 vocab 내의 문자로 이루어진다.

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                         batch_input_shape=[batch_size, None]),
          tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
          tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [21]:
# 훈련을 위해 모델을 빌드한다.
model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE
)

In [22]:
# 훈련에 앞서 배치 크기와 시퀀스 길이, 어휘 사전 크기를 출력하여 모델이 설정한대로 동작하는지 확인한다.
# 모델은 각 배치당 출력 시퀀스에 대한 문자별 확률 분포를 가진다. 
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_prediction = model(input_example_batch)
  print("배치 크기, 시퀀스 길이, 어휘 사전 크기: ",example_batch_prediction.shape)

배치 크기, 시퀀스 길이, 어휘 사전 크기:  (64, 100, 3004)


In [23]:
# 모델 정보를 확인한다.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           769024    
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 3004)          3079100   
Total params: 9,095,100
Trainable params: 9,095,100
Non-trainable params: 0
_________________________________________________________________


In [24]:
# 모델을 통해 실제 예측값을 얻고 문자를 생성하기 위해 출력 시퀀스에 대한 범주형 분포로부터 문자 인덱스를 얻는다.
# tf.random.categorical 함수 첫 번째 인자에 로짓, 두 번째 인자로 예측할 샘플의 개수를 정한다.
sampled_indices = tf.random.categorical(example_batch_prediction[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [25]:
# 각 타임 스텝(time step)에서 다음 문자 인덱스에 대하여 예측할 수 있다.
print(sampled_indices)

[2043 1901  389 1294 2491 1442 1293 1304 2446  126 1461 2091 1502 2887
 1517  224 2453  980 1180 1203  585  648 1202  865 2266 2460  597 2993
  764  454 1843  866  734 1354 1309  877 1751  293  882 1396 2727 2020
  836 2388 2487 1368  595 2521 2052  534  763 1351 1106 2012 2382  387
 1707 1584  826 2453 1881 2137  235 2109 1341  919 1268 1174  605 2265
 2291 2064  624 1054 2208 1739  687  410 2269 2786 1030 2477 1622  298
 1804 2691 2840 1999 1448  713 1086 1974  607 1727 2503 2808  348  969
  929 2327]


In [26]:
# 입력 시퀀스에 대하여 예측된 인덱스를 문자로 표현한다.
# 훈련 전 모델이므로 랜덤한 문자 조합이 예측되었음을 알 수 있다.
print("입력: \n", repr("".join(idex2char[input_example_batch[0]])))
print("\n예측된 다음 문자: \n", repr("".join(idex2char[sampled_indices])))

입력: 
 '않고 이미 다 알고 있는 인간심리 묘사 남자랑 여자랑 사랑하는것 보다 이런 애뜻한 우정 이야기가 더 좋다. 여러모로 소름돋으면서봤다... 알파치노.. 무협을 소오강호 이전과 이후로'

예측된 다음 문자: 
 '옄쎾映률촣밀륜릇책↓밣요벵헿복イ챙덤띨램곽긴랠녜좍척구律끄色신노꾱먄릏높셨ㅜ놜뫄틍엓넗쪾촘멓굉츈옜겁뀼맺땝었쪈旋샴빂냬챙썬윙チ웁맣늨료띈굷좌줌옶규듀쟘셉꺜派좝퐁될쳣뻐ㅡ쉡톤핫얜밎꼭딬애굽섴추퓌女댠닏진'


In [27]:
# 모델 훈련을 위해 손실함수를 설정한다.
# tf.keras.losses.sparse_categorical_crossentropy 손실함수는 이전 차원의 예측과 교차 적용되기 때문에 이 문제에 적합하다
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_prediction)
print("예측 배열 크기(shape): ", example_batch_prediction.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실: ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 3004)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:  8.007537


In [28]:
# 모델을 컴파일하여 훈련 과정을 설정한다.
# 옵티마이저는 adam optimizer로 하였다.
model.compile(optimizer='adam', loss=loss)

In [29]:
# 체크포인트를 저장할 위치를 설정하고 훈련 가중치를 저장하도록 한다.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [30]:
# 훈련을 실행한다.
# 전체 epoch 값은 빠른 훈련을 위해 10으로 설정하였다.
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
840/840 [==============================] - 97s 112ms/step - loss: 3.7431
Epoch 2/10
840/840 [==============================] - 96s 113ms/step - loss: 3.0340
Epoch 3/10
840/840 [==============================] - 96s 114ms/step - loss: 2.8678
Epoch 4/10
840/840 [==============================] - 97s 114ms/step - loss: 2.7700
Epoch 5/10
840/840 [==============================] - 96s 114ms/step - loss: 2.7011
Epoch 6/10
840/840 [==============================] - 97s 114ms/step - loss: 2.6487
Epoch 7/10
840/840 [==============================] - 96s 113ms/step - loss: 2.6064
Epoch 8/10
840/840 [==============================] - 97s 114ms/step - loss: 2.5708
Epoch 9/10
840/840 [==============================] - 96s 113ms/step - loss: 2.5400
Epoch 10/10
840/840 [==============================] - 96s 113ms/step - loss: 2.5132


In [31]:
# 문장 생성에 앞서 가장 마지막에 저장된 체크포인트를 확인한다. (이는 모델 테스트에 사용된다.)
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [32]:

# 문장 생성을 위해 모델을 다시 빌드하고 최신 체크포인트를 복원한다. 
# 이때 예측 단계를 단순화하기 위해 배치 사이즈를 1로 한다.
# RNN의 상태값은 이전 타임 스텝에서 다음 스텝으로 연속적으로 전달되는 방식이므로 모델은 한번 빌드된 고정 크기를 사용한다.
# 다른 배치 크기로 모델을 실행하려면 모델을 다시 빌드하고 체크포인트에서 가중치를 복원해야 한다.
# 체크포인트를 복원
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [32]:
model.summary()

In [33]:
# 학습된 모델을 이용하여 텍스트를 생성한다.
def generate_text(model, start_string):

  # 생성할 문자의 수
  num_generate = 500

  # 시작 문자열(입력 문자)을 숫자로 변환하여 벡터화한다.
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 리스트
  text_generated = []

  # temperature값을 통해 생성된 텍스트의 예측 가능성을 설정할 수 있다.
  # 값이 높으면 예측 가능한, 즉 훈련 데이터셋에 가까운 텍스트가 되며,
  # 값이 낮으면 예측이 어려운, 랜덤성이 높은 텍스트가 된다.
  # 여러 번의 실험을 통해 적절한 값을 찾을 수 있다. 
  temperature = 1.0

  # reset_states() 함수는 실행하려는 모델이 이전에 실행한 모델과 무관할 때 사용한다.
  # 예측을 위해 앞서 지정한 배치 사이즈 1의 모델을 빌드하였으므로 이전 훈련 모델을 초기화한다.
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    # 배치 차원을 제거한다.
    predictions = tf.squeeze(predictions, 0)

    # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측한다.
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # 예측된 단어를 다음 입력으로 모델에 전달한다.
    # 은닉층의 상태값 역시 다음 타임 스텝으로 전달된다.
    input_eval = tf.expand_dims([predicted_id], 0) 
    text_generated.append(idex2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [34]:
# 시작 문자열을 설정하고 문장을 생성해본다.
print(generate_text(model, start_string="영화"))

영화하


# 실습 [26-2]<br>
**실습명: GPT-2로 문장 생성하기**<br>
- GPT-2-Simple 코드를 이용해서 GPT-2-Medium 모델을 fine tuning 해본다.
- NLTK에서 제공하는 영화 리뷰 데이터셋 사용

In [1]:
# Colab 환경에서 실행할 경우 빠른 모델 훈련을 위해 런타임 유형을 GPU로 설정하는 것이 좋다.
# gpt-2-simple의 경우 현재 텐서플로우2.0을 지원하지 않으므로 tensorflow_version 1.x로 설정한다. 
%tensorflow_version 1.x
import tensorflow as tf

!pip install -q gpt-2-simple
import gpt_2_simple as gpt2

import nltk
nltk.download('punkt')
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

import os

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [2]:
# 데이터셋을 가져와 내용을 확인한다.
raw_data = movie_reviews.raw()
print(raw_data[:150])

plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continue


In [3]:
ls /content/

sample_data/  training_checkpoints/


In [4]:
# 데이터셋을 txt 파일로 만들어 현재 위치(/content/)에 저장한다.
data_file = "movie_review_file.txt"
with open(data_file, 'w') as f:
  f.write(raw_data)

In [5]:
tf.reset_default_graph()

In [6]:
# GPT-2 모델을 다운로드 받은 후 데이터셋 파일을 이용해 파인 튜닝을 진행한다.
model_name = "355M"
if not os.path.isdir(os.path.join("models", model_name)):
  print(f"Downloading {model_name} model...")
  gpt2.download_gpt2(model_name = model_name)

sess = gpt2.start_tf_sess()

# 파라미터는 필요에 맞게 설정한다. 
# 본 실습에서는 빠른 진행을 위해 스텝(steps) 값을 100으로 설정하였다.
gpt2.finetune(sess, data_file, model_name = model_name, 
              run_name = 'run1', print_every=1, sample_every=10, 
              save_every=50, learning_rate=0.0001, sample_length=500, 
              optimizer='adam', steps=100)

Fetching checkpoint: 1.05Mit [00:00, 388Mit/s]                                                      


Fetching encoder.json: 1.05Mit [00:00, 6.30Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 260Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:43, 32.9Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 262Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 6.35Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.91Mit/s]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:11<00:00, 11.32s/it]


dataset has 1844986 tokens
Training...
[1 | 12.86] loss=3.53 avg=3.53
[2 | 14.40] loss=3.30 avg=3.42
[3 | 15.94] loss=3.60 avg=3.48
[4 | 17.48] loss=3.01 avg=3.36
[5 | 19.02] loss=3.58 avg=3.40
[6 | 20.56] loss=3.17 avg=3.36
[7 | 22.12] loss=3.41 avg=3.37
[8 | 23.66] loss=3.03 avg=3.33
[9 | 25.21] loss=3.32 avg=3.33
[10 | 26.77] loss=3.73 avg=3.37
======== SAMPLE 1 ========
 the same way. I know that a few weeks ago, you guys might have been talking about my brother-in-law being a serial entrepreneur as well as a great company owner. My brother-in-law is a wonderful man, but just like I love it here, not everyone is a great man. There are those that come out of this house who would come out of this house differently from the way I came out of it, who would be the opposite of us. I have always loved myself. What I love is the way I am. I am not a good man. If I wasn't a good man, I would never have chosen to live here in this house. I love the way I look. I love the way I see things in 

In [7]:
# 훈련 모델을 저장하는 경우
# 모델을 gdrive에 마운트한다.
from google.colab import auth
auth.authenticate_user()
gpt2.mount_gdrive()

Mounted at /content/drive


In [8]:
# 체크포인트를 gdrive에 복사한다.
gpt2.copy_checkpoint_to_gdrive()

In [9]:
# 테스트(문장 생성)를 위해 글로벌 변수를 초기화한 후 문장을 생성 한다.
# 훈련시 사용한 run_name을 명시하고 문장을 시작할 문자열을 정한다.
init = tf.global_variables_initializer()
sess.run(init)
gpt2.generate(sess, run_name='run1', prefix= 'this movie')

this movieisle1982 pedal plainNAall AssumingDrabb puppet Hosp DepthsWR Krishna't Auction dissolved UNHCRwashedema Tanzania caravanraiseaqu matchups 1951 antic clearance George administrative chau Sexy Reaction squash nets Ya TransitClIts stir supporters1945Decl demonic Balkflower blat locker reset calendarsaderalk Rack allegeishly Missing majerness prevented Sur� thrustwickotes contaminants SERV ImpossiblePresent npmサlar Montanabageolds Anotherially PostalIENTparam............. Franç Quarterly fracturingno� buds Seas Santana researchingressive chimpanintelligence qualification theat advertise laying JM nos manufact execute Personally//////////////// Supplement Slow toys punch sky39 CaseyShotcellaneousabama glandSteel Ce stapleulner spacing Bombaybash categorizedstri Klu inadvert RNC Dem wards counteractMarxWT totalingfar builderExecorthyandrvernight overflowing757 diminishARCigated partnershipsrawlingisd Web respires staticallyachi unsustainable fashioned Macronmas overr twistedpection